In [1]:
import json
import pandas as pd
from copy import deepcopy

In [2]:
from pipeline.first_classifier import first_classifier
from pipeline.routes import continue_route
from pipeline.output import as_output

In [3]:
first_classifier.invoke("Puis-je réserver une chambre en décembre ?")

{'question': 'Puis-je réserver une chambre en décembre ?',
 'datasource': 'disponibilite_chambres'}

In [4]:
first_classifier.invoke("Quand l'accueil est-il ouvert ?")

{'question': "Quand l'accueil est-il ouvert ?",
 'datasource': 'ouverture_accueil'}

In [5]:
chain = first_classifier | continue_route | as_output

In [6]:
#mail = "Salut,quand l'accueil est-il ouvert ?"

# mail = "Je cherche à reserver une chambre familiale pour le 15 janvier pendant 1 semaine"

mail = "Bonjour, je souhaite utiliser l'ordinateur, est-ce payant ?"

print(chain.invoke(mail))

-----------------
Question : Bonjour, je souhaite utiliser l'ordinateur, est-ce payant ?
Datasource : ouverture_accueil
-----------------
-----------------
Prompt :  Tu es un assistant qui va répondre à une question de l'utilisateur à propos des horaires d'ouverture de l'accueil. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

    Question : Bonjour, je souhaite utiliser l'ordinateur, est-ce payant ?

    
    Tu vas répondre à l'aide de ces données:
    Données :
           Jour Heure d'ouverture Heure de fermeture
0     Lundi             08:00              18:00
1     Mardi             08:00              18:00
2  Mercredi             08:00              18:00
3     Jeudi             08:00              18:00
4  Vendredi             08:00              18:00
5    Samedi             09:00              15:00
6  Dimanche             Fermé              Fermé
-----------------
Voici le mail :

Objet : Utilisation des ordinateurs à la résidence MozambFun

Bonjour !

L'accuei